In [4]:
%matplotlib inline
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
import os
from IPython.display import display

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout
import plotly.plotly as py
init_notebook_mode(connected=True)

In [13]:
disk_engine = create_engine('sqlite:///C:/Data/Data/samspill.db') 

In [5]:
def getDir(dir = ''):
    directory = r'C:/Data/Repos/Samspill/Dataset/'
    if len(dir) > 0:
        directory = directory + dir + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    return directory

In [7]:
fil = r'C:\Data\Repos\samspill\dataset\SIVA_Målbedrifter i inkubatorene 2013-2015_til NFR_versjon 2 med unike id og kun aktive bedrifter i rapportåret.xlsx'
excel = pd.ExcelFile(fil,encoding='utf-8')
excel.sheet_names

['Data', 'Forklaringer']

In [10]:
i_bedr = excel.parse('Data')
i_bedr.head(1)

,Rapportår,Inkubator,InkubatorID,Org.nr_INK,Navn på bedrift i inkubasjon,MålbedriftID,Orgnr_bedrift,Fase pr 31.12,I Forr.messig innovasjon i løpet av året,Inkubatorbedrift i løpet av året,...,Kommune,Fylke,Markedsverdien på innovasjonsstøttetjenester fra inkubatoren,Fakturert egenandel,Sum innovasjonsstøtte fra inkubatoren,Støtteintensitet,Kvinnelig daglig leder,Kvinnelig styreleder,Kvinnelig Grunder,Kvinnebedrift
0,2013,7Sterke AS,19.0,997144368.0,ELMICO AS,539.0,985338671,Inkubatorbedrift,0.0,1.0,...,SØR-ODAL,HEDMARK,301710,103478,198232,65.7,1,0,1,1.0


In [15]:
i_bedr.to_sql('siva_i_b', disk_engine, if_exists='replace')

In [11]:
fil = r'C:\Data\Repos\samspill\dataset\SIVA_Målbedrifter i næringshagene 2013-2015_til NFR_versjon 2 med unike id.xlsx'
excel = pd.ExcelFile(fil,encoding='utf-8')
excel.sheet_names

['Data', 'Forklaringer']

In [12]:
n_bedr = excel.parse('Data')
n_bedr.head(1)

,Rapportår,Næringshage,NæringshageID,Org-nr NH,Målbedrift,BedriftId,Orgnummer,Registeringstype,Etableringsdato,Avtalestart,...,Kommune,Fylke,MarkedsverdienPåInnovasjonsstøttetjenesterFraNH,SumInnovasjonsstøtteFraNH,FakturertEgenandel,Støtteintensitet,Kvinnelig daglig leder,Kvinnelig styreleder,Kvinnelig gründer,Kvinnebedrift
0,2013,Aura Næringshage AS,23.0,998423481.0,3S Prosjekt avd Sunndal,1821.0,986580719,Målbedrift,2004-03-15,2012-01-01,...,SUNNDAL,MØRE OG ROMSDAL,95600,66800.0,28800,69.87,0,0,0,0


In [16]:
n_bedr.to_sql('siva_n_b', disk_engine, if_exists='replace')

In [ ]:
sql = """
    SELECT * FROM n_bedr AS i LEFT JOIN (
    SELECT e.orgnr,
           e.Juridisk_selskapsnavn,
           e.STIFTDATO,
           e.REGDATO,
           e.SLETDATO,
           e.Fylke,
           e.Bransje,
           e.Antall_ansatte,
           e.Antall_ansatte_konsern,
           e.ANTALL_AARSVERK,
           e.ANTALL_AARSVERK_KONSERN,
           e.Regnaar, 
           e.LATITUDE, 
           e.LONGITUDE, 
           e.SEKTOR_2014 AS 'Sektor' 
    FROM eniro AS e
    WHERE e.Regnaar IN (SELECT MAX(Regnaar) from eniro GROUP BY orgnr))
    ON i.orgnummer = e.orgnr
    """

